As noted in Tutorial 00, `pygsti` can provide many reports or presentation slides.

In this tutorial, we look at how to customize some of the output of `pygsti`'s report generating module.

In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
#Import the GST module -- you probably want this at the beginning of every notebook
import pygsti
import json

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

Creating reports is a method of the `pygsti.report.Results` object. Below, we create such an object, and then initialize it with the appropriate fiducials, germs, maximum lengths, etc.

It's important to note that the `pygsti.report.Results` object requires an argument to indicate which objective function you wish to score the estimate using. It's recommended that, if you ran MC2GST, you use the `"chi2"` argument, while if you ran MLGST, you use `"logL"`. Otherwise, you will score your estimate based on an objective function you weren't optimizing over, so your scores may be rather weird!

First, we'll make smaller reports that don't contain any confidence intervals for the estimates,
nor any appendices.

NOTE: In order for you to create PowerPoint files, you'll have to have `python-pptx` installed on your system.

In [3]:
# Follow Algorithm tutorial to generate LSGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
gs_target.set_basis("gm",2)
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.gaugeopt_to_target(gs_lgst, gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of MC2GST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))
lsgstListOfLists = [ pygsti.io.load_gatestring_list("tutorial_files/Example_LSGSTlist%d.txt" % l) for l in maxLengthList]
        
gs_lsgst_list = pygsti.do_iterative_mlgst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClip=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )

go_gatesets = [ pygsti.gaugeopt_to_target(gs, gs_target,itemWeights={'gates': 1, 'spam': 0.001})
               for gs in gs_lsgst_list]

Loading from cache file: tutorial_files/Example_Dataset.txt.cache
--- LGST ---
--- Iterative MLGST: Iter 01 of 10  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs each, to distribute over 56 params (taken as 1 param groups of ~56 params).
  Sum of Chi^2 = 40.9238 (92 data params - 40 model params = expected mean of 52; p-value = 0.866034)
  Completed in 0.1s
  2*Delta(log(L)) = 41.1104
  Iteration 1 took 0.1s
  
--- Iterative MLGST: Iter 02 of 10  92 gate strings ---: 
  --- Minimum Chi^2 GST ---
  Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
   groups of ~1 procs each, to distribute over 56 params (taken as 1 param groups of ~56 params).
  Sum of Chi^2 = 40.9238 (92 data params - 40 model params = expected mean of 52; p-value = 0.866034)
  Completed in 0.0s
  2*Delta(log(L)) = 41.1104
  Iteration 2 took 0.0s
  
--- Iterative M

In [4]:
res = pygsti.report.Results()
res.init_Ls_and_germs("logl", gs_target, ds, gs_clgst, maxLengthList, germList,
                    go_gatesets, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False)

In [5]:
#Make smaller reports (no confidence intervals or appendicies)
res.create_full_report_pdf(filename="tutorial_files/Example_reportB.pdf", verbosity=2, confidenceLevel=None,
                        debugAidsAppendix=False, gaugeOptAppendix=False,
                        pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_brief_report_pdf(filename="tutorial_files/Example_briefB.pdf", verbosity=2, confidenceLevel=None)

res.create_presentation_pdf(filename="tutorial_files/Example_slidesB.pdf", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_presentation_ppt(filename="tutorial_files/Example_slidesB.ppt", verbosity=2, confidenceLevel=None,
                         debugAidsAppendix=False, pixelPlotAppendix=False, whackamoleAppendix=False)

res.create_general_report_pdf(filename="tutorial_files/Example_generalB.pdf", verbosity=2, confidenceLevel=None)

*** Generating tables ***
 Iter 01 of 17 :   Generating table: targetSpamTable  [0.0s]
 Iter 02 of 17 :   Generating table: targetGatesTable  [0.0s]
 Iter 03 of 17 :   Generating table: datasetOverviewTable  [0.0s]
 Iter 04 of 17 :   Generating table: bestGatesetSpamTable  [0.0s]
 Iter 05 of 17 :   Generating table: bestGatesetSpamParametersTable  [0.0s]
 Iter 06 of 17 :   Generating table: bestGatesetGaugeOptParamsTable  [0.0s]
 Iter 07 of 17 :   Generating table: bestGatesetGatesTable  [0.0s]
 Iter 08 of 17 :   Generating table: bestGatesetChoiTable  [0.0s]
 Iter 09 of 17 :   Generating table: bestGatesetDecompTable  [0.0s]
 Iter 10 of 17 :   Generating table: bestGatesetRotnAxisTable  [0.0s]
 Iter 11 of 17 :   Generating table: bestGatesetVsTargetTable  [0.2s]
 Iter 12 of 17 :   Generating table: bestGatesetErrorGenTable  [0.0s]
 Iter 13 of 17 :   Generating table: fiducialListTable  [0.0s]
 Iter 14 of 17 :   Generating table: prepStrListTable  [0.0s]
 Iter 15 of 17 :   Generating t

In [6]:
res.create_full_report_pdf(filename="tutorial_files/Example_report.pdf", verbosity=2, confidenceLevel=95,
                        debugAidsAppendix=True, gaugeOptAppendix=True,
                        pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_brief_report_pdf(filename="tutorial_files/Example_brief.pdf", verbosity=2, confidenceLevel=95)

res.create_presentation_pdf(filename="tutorial_files/Example_slides.pdf", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_presentation_ppt(filename="tutorial_files/Example_slides.ppt", verbosity=2, confidenceLevel=95,
                         debugAidsAppendix=True, pixelPlotAppendix=True, whackamoleAppendix=True)

res.create_general_report_pdf(filename="tutorial_files/Example_generalB.pdf", verbosity=2, confidenceLevel=95)

    
--- Hessian Projector Optimization for gate CIs (L-BFGS-B) ---
  203s           0.0264001138
  219s           0.0260997631
  226s           0.0257551198
  234s           0.0256330228
  242s           0.0254585068
  250s           0.0254109392
  258s           0.0253685639
  266s           0.0253497042
  274s           0.0253435862
  282s           0.0253401520
  290s           0.0253379820
  298s           0.0253362146
  306s           0.0253341328
  314s           0.0253318189
  322s           0.0253296326
  330s           0.0253281099
  338s           0.0253269177
  346s           0.0253257997
  353s           0.0253247547
  361s           0.0253236837
  369s           0.0253226634
  The resulting min sqrt(sum(gateCIs**2)): 0.025322
*** Generating tables ***
 Iter 01 of 17 :   Generating table: targetSpamTable (w/95% CIs)  [0.0s]
 Iter 02 of 17 :   Generating table: targetGatesTable (w/95% CIs)  [0.0s]
 Iter 03 of 17 :   Generating table: datasetOverviewTable (w/95% CIs)  [0.0s]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib-1.5.3-py3.5-macosx-10.6-intel.egg/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


  [36.5s]
 Iter 3 of 3 :   Generating figure: whackGiMoleBoxes (w/95% CIs)  Generating figure: whackGiMoleBoxes  [35.9s]
 Iter 1 of 3 :   Generating figure: whackGxMoleBoxesSummed (w/95% CIs)  Generating figure: whackGxMoleBoxesSummed  [22.4s]
 Iter 2 of 3 :   Generating figure: whackGyMoleBoxesSummed (w/95% CIs)  Generating figure: whackGyMoleBoxesSummed  [22.4s]
 Iter 3 of 3 :   Generating figure: whackGiMoleBoxesSummed (w/95% CIs)  Generating figure: whackGiMoleBoxesSummed  [22.0s]

*** Merging into template file ***
Latex file(s) successfully generated.  Attempting to compile with pdflatex...
Initial output PDF tutorial_files/Example_report.pdf successfully generated.
Final output PDF tutorial_files/Example_report.pdf successfully generated. Cleaning up .aux and .log files.
*** Generating tables ***
  Retrieving cached table: bestGatesetSpamTable (w/95% CIs)
  Retrieving cached table: bestGatesetSpamParametersTable (w/95% CIs)
  Retrieving cached table: bestGatesetGatesTable (w/95%

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib-1.5.3-py3.5-macosx-10.6-intel.egg/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


  [2.3s]
  Retrieving cached table: datasetOverviewTable (w/95% CIs)
  Generating table: bestGatesetEvalTable (w/95% CIs)  [0.6s]
  Generating table: bestGatesetRelEvalTable (w/95% CIs)  [0.6s]
  Generating table: targetGatesBoxTable (w/95% CIs)  [1.3s]
  Generating table: bestGatesetGatesBoxTable (w/95% CIs)  [2.7s]
  Generating table: bestGatesetErrGenBoxTable (w/95% CIs)  [3.5s]
  Retrieving cached table: fiducialListTable (w/95% CIs)
  Retrieving cached table: prepStrListTable (w/95% CIs)
  Retrieving cached table: effectStrListTable (w/95% CIs)
  Generating table: germList2ColTable (w/95% CIs)  [0.0s]
  Retrieving cached table: progressTable (w/95% CIs)
*** Generating plots *** (16.5s elapsed)
 -- LogL plots:  Iter 1 of 3 :   Retrieving cached figure: colorBoxPlotKeyPlot (w/95% CIs)
 Iter 2 of 3 :   Generating figure: bestEstimateSummedColorBoxPlot (w/95% CIs)  Retrieving cached figure: bestEstimateSummedColorBoxPlot
 Iter 3 of 3 :   Generating special: bestEstimateColorBoxPlotPag

In [7]:
#Results can be pickled after generating reports in order to save cached derived quantities
import pickle
pickle.dump(res, open("tutorial_files/example_results.pkl","wb"))

In [8]:
import pygsti.report.latex as LU
from imp import reload
reload(LU)
LU.latex_value(1.2e30,2)

'1\\e{30}'